<a href="https://colab.research.google.com/github/vatsal-kr/LegalNER-CS60075/blob/main/Legal_NER_DataFormatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import json
import os
import sys
import nltk
import bisect
import re
import spacy
from collections import Counter, defaultdict
from spacy.training import offsets_to_biluo_tags
nltk.download('punkt')
!python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")
# sys.path.append('/content/drive/My Drive/LegalEval/')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-21 19:21:32.480313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-21 19:21:32.480461: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-21 19:21:32.480487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
!unzip '/content/drive/My Drive/LegalEval/NER_TRAIN.zip'
!unzip '/content/drive/My Drive/LegalEval/NER_DEV.zip'

Archive:  /content/drive/My Drive/LegalEval/NER_TRAIN.zip
  inflating: NER_TRAIN_JUDGEMENT.json  
  inflating: NER_TRAIN_PREAMBLE.json  
Archive:  /content/drive/My Drive/LegalEval/NER_DEV.zip
   creating: NER_DEV/
  inflating: NER_DEV/.DS_Store       
  inflating: __MACOSX/NER_DEV/._.DS_Store  
  inflating: NER_DEV/NER_DEV_PREAMBLE.json  
   creating: NER_DEV/.ipynb_checkpoints/
  inflating: NER_DEV/NER_DEV_JUDGEMENT.json  
  inflating: NER_DEV/.ipynb_checkpoints/CYCLE1_FINAL_REVIEW_COMBINED_29_AUG-checkpoint.json  
  inflating: NER_DEV/.ipynb_checkpoints/CYCLE1_FINAL_REVIEW_JUDGEMENT_29_AUG-checkpoint.json  


In [ ]:
judgements_df = pd.read_json('NER_TRAIN_JUDGEMENT.json')
preamble_df = pd.read_json('NER_TRAIN_PREAMBLE.json')

In [ ]:
judgement_texts = judgements_df['data'].str.get('text')
preamble_texts = preamble_df['data'].str.get('text')

In [ ]:
judgement_labels = judgements_df['annotations'].str[0].str.get('result')
judgement_labels = [[label.get('value') for label in all_labels] for all_labels in judgement_labels]
preamble_labels = preamble_df['annotations'].str[0].str.get('result')
preamble_labels = [[label.get('value') for label in all_labels] for all_labels in preamble_labels]

In [ ]:
def generate_tokenized_text_and_labels(texts, labels):
  tokenized_texts, bio_labels = [], []
  for text, label in zip(texts, labels):
    entities = []
    for each_label in label:
      entities.append((each_label['start'], each_label['end'], each_label['labels'][0]))
    doc = nlp(text)
    bio_labels.append(offsets_to_biluo_tags(doc, entities))

    tokenized_texts.append([x.text for x in doc])

  return tokenized_texts, bio_labels

In [ ]:
judgement_texts_tokenized, judgement_labels_tagged = generate_tokenized_text_and_labels(judgement_texts, judgement_labels)
preamble_texts_tokenized, preamble_labels_tagged = generate_tokenized_text_and_labels(preamble_texts, preamble_labels)

In [ ]:
judgements_data = pd.concat([pd.Series(judgement_texts, name='text'), pd.Series(judgement_texts_tokenized, name='text_tokenized'), pd.Series(judgement_labels_tagged, name='label')], axis=1)
preamble_data = pd.concat([pd.Series(preamble_texts, name='text'), pd.Series(preamble_texts_tokenized, name='text_tokenized'), pd.Series(preamble_labels_tagged, name='label')], axis=1)

In [ ]:
judgements_data['label'] = judgements_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[U*L*]+-', 'I-', x), lst)))
preamble_data['label'] = preamble_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[U*L*]+-', 'I-', x), lst)))

In [ ]:
judgements_data['label'] = judgements_data['label'].apply(lambda lst: list(map(lambda x: 'O' if x=='-' else x, lst)))
preamble_data['label'] = preamble_data['label'].apply(lambda lst: list(map(lambda x: 'O' if x=='-' else x, lst)))

In [ ]:
# judgements_data['label'] = judgements_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[\([{\'})\]]', '', x), lst)))
# preamble_data['label'] = preamble_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[\([{\'})\]]', '', x), lst)))

In [ ]:
judgements_data.to_pickle('/content/drive/My Drive/LegalEval/judgement_data_train.pkl')
preamble_data.to_pickle('/content/drive/My Drive/LegalEval/preamble_data_train.pkl')

## Build Vocabulary of words and tags

In [ ]:
min_count_word = 1
min_count_tag = 1
pad_word = '<pad>'
pad_tag = 'O'
unk_word = 'UNK'
words_vocab, tags_vocab = Counter(), Counter()

text_lines = list(judgements_data['text'])
for line in text_lines:
  words_vocab.update(line.strip().split(' '))
text_lines = list(preamble_data['text'])
for line in text_lines:
  words_vocab.update(line.strip().split(' '))

label_lines = [' '.join(judgements_data['label'][i]) for i in range(len(judgements_data))]
for line in label_lines:
  tags_vocab.update(line.strip().split(' '))
label_lines = [' '.join(preamble_data['label'][i]) for i in range(len(preamble_data))]
for line in label_lines:
  tags_vocab.update(line.strip().split(' '))

In [ ]:
words = [tok for tok, count in words_vocab.items() if count >= min_count_word]
tags = [tok for tok, count in tags_vocab.items() if count >= min_count_tag]

# Add pad tokens
if pad_word not in words: words.append(pad_word)
if pad_tag not in tags: tags.append(pad_tag)

# add word for unknown words 
words.append(unk_word)

In [ ]:
with open('/content/drive/My Drive/LegalEval/words.txt', "w") as f:
  for token in words:
    f.write(token + '\n')
with open('/content/drive/My Drive/LegalEval/tags.txt', "w") as f:
  for token in tags:
    f.write(token + '\n')

## Dev Data

In [ ]:
judgements_df = pd.read_json('NER_DEV/NER_DEV_JUDGEMENT.json')
preamble_df = pd.read_json('NER_DEV/NER_DEV_PREAMBLE.json')

In [ ]:
judgement_texts = judgements_df['data'].str.get('text')
preamble_texts = preamble_df['data'].str.get('text')

In [ ]:
judgement_labels = judgements_df['annotations'].str[0].str.get('result')
judgement_labels = [[label.get('value') for label in all_labels] for all_labels in judgement_labels]
preamble_labels = preamble_df['annotations'].str[0].str.get('result')
preamble_labels = [[label.get('value') for label in all_labels] for all_labels in preamble_labels]

In [ ]:
def generate_tokenized_text_and_labels(texts, labels):
  tokenized_texts, bio_labels = [], []
  for text, label in zip(texts, labels):
    entities = []
    for each_label in label:
      entities.append((each_label['start'], each_label['end'], each_label['labels'][0]))
    doc = nlp(text)
    bio_labels.append(offsets_to_biluo_tags(doc, entities))

    tokenized_texts.append([x.text for x in doc])

  return tokenized_texts, bio_labels

In [ ]:
judgement_texts_tokenized, judgement_labels_tagged = generate_tokenized_text_and_labels(judgement_texts, judgement_labels)
preamble_texts_tokenized, preamble_labels_tagged = generate_tokenized_text_and_labels(preamble_texts, preamble_labels)

/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "This will be clarified in the instant case by comp..." with entities "[(60, 127, 'PROVISION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "

Digitally signed by:RAJENDER SINGH KARKI Signing..." with entities "[(22, 42, 'OTHER_PERSON'), (56, 66, 'DATE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "It is seen from the 

In [ ]:
judgements_data = pd.concat([pd.Series(judgement_texts, name='text'), pd.Series(judgement_texts_tokenized, name='text_tokenized'), pd.Series(judgement_labels_tagged, name='label')], axis=1)
preamble_data = pd.concat([pd.Series(preamble_texts, name='text'), pd.Series(preamble_texts_tokenized, name='text_tokenized'), pd.Series(preamble_labels_tagged, name='label')], axis=1)

In [ ]:
judgements_data['label'] = judgements_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[U*L*]+-', 'I-', x), lst)))
preamble_data['label'] = preamble_data['label'].apply(lambda lst: list(map(lambda x: re.sub('[U*L*]+-', 'I-', x), lst)))

In [ ]:
judgements_data['label'] = judgements_data['label'].apply(lambda lst: list(map(lambda x: 'O' if x=='-' else x, lst)))
preamble_data['label'] = preamble_data['label'].apply(lambda lst: list(map(lambda x: 'O' if x=='-' else x, lst)))

In [ ]:
judgements_data.to_pickle('/content/drive/My Drive/LegalEval/judgement_data_dev.pkl')
preamble_data.to_pickle('/content/drive/My Drive/LegalEval/preamble_data_dev.pkl')

In [ ]:
# judgements_data.iloc[16]['label']

In [ ]:
# judgement_labels[37]

[{'start': 22,
  'end': 42,
  'text': 'RAJENDER SINGH KARKI',
  'labels': ['OTHER_PERSON']},
 {'start': 56, 'end': 66, 'text': '13.01.2021', 'labels': ['DATE']}]

In [ ]:
# for x in nlp(judgement_texts[37]):
#   print(x)




Digitally
signed
by
:
RAJENDER
SINGH
KARKI
Signing
Date:13.01.2021
18:54:39
 
31
.
Claimant(s
)
to
respond
to
the
offer
of
the
Insurance
Company
within
30
days


In [ ]:
# text = judgement_texts[16]
# doc = nlp(text)
# entities = [(60, 127, "['PROVISION']")]
# offsets_to_biluo_tags(nlp.make_doc(text), entities)

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']